# Open AI မော်ဒယ်များကို Fine Tuning ပြုလုပ်ခြင်း

ဒီ notebook ကို Open AI ရဲ့ [Fine Tuning](https://platform.openai.com/docs/guides/fine-tuning?WT.mc_id=academic-105485-koreyst) လမ်းညွှန်ချက်အရ ရေးသားထားပါတယ်။

Fine-tuning ဆိုတာ သင့်ရဲ့ application အတွက် foundation model တစ်ခုကို ထပ်မံလိုအပ်တဲ့ data နဲ့ context တွေဖြင့် ပြန်လည်လေ့ကျင့်ခြင်းဖြစ်ပြီး၊ အထူးသဖြင့် သတ်မှတ်ထားတဲ့ use case သို့မဟုတ် scenario အတွက် ပိုမိုကောင်းမွန်စေပါတယ်။ _few shot learning_ နဲ့ _retrieval augmented generation_ လို prompt engineering နည်းလမ်းတွေကလည်း default prompt ကို သက်ဆိုင်ရာ data တွေနဲ့ တိုးတက်အောင်လုပ်နိုင်ပေမယ့်၊ ဒီနည်းလမ်းတွေမှာ သုံးမယ့် foundation model ရဲ့ max token window size ကန့်သတ်ချက်ရှိပါတယ်။

Fine-tuning နဲ့ဆိုရင်တော့ မော်ဒယ်ကိုလိုအပ်တဲ့ data တွေနဲ့ တကယ့်ကို ပြန်လည်လေ့ကျင့်တာဖြစ်လို့ (max token window ထဲမှာ မတင်နိုင်တဲ့ ဥပမာတွေကိုပါ သုံးနိုင်ပါတယ်) - နောက်ထပ် _custom_ မော်ဒယ်တစ်ခုကို deploy လုပ်နိုင်ပါတယ်။ ဒီ custom မော်ဒယ်မှာတော့ inference အချိန်မှာ ဥပမာတွေ ထပ်မပေးရတော့ပါဘူး။ ဒါကြောင့် prompt design ကို ပိုမိုထိရောက်စေသလို (token window ကို အခြားအရာတွေအတွက် ပိုအသုံးချနိုင်ပါတယ်)၊ တစ်ချို့အခါမှာတော့ ကုန်ကျစရိတ်ကိုပါ လျှော့ချနိုင်ပါတယ် (inference အချိန်မှာ မော်ဒယ်ကို ပို့ရမယ့် token အရေအတွက် လျှော့နည်းသွားလို့)။

Fine tuning လုပ်ရာမှာ လုပ်ဆောင်ရမယ့် လေးဆင့်ရှိပါတယ် -
1. လေ့ကျင့်မယ့် data ကို ပြင်ဆင်ပြီး upload လုပ်ပါ။
1. Training job ကို run လုပ်ပြီး fine-tuned model ရယူပါ။
1. Fine-tuned model ကို အရည်အသွေးအတွက် စစ်ဆေးပြီး တိုးတက်အောင် ပြန်လုပ်ပါ။
1. စိတ်တိုင်းကျလာတဲ့အခါ Fine-tuned model ကို inference အတွက် deploy လုပ်ပါ။

Foundation model အားလုံးမှာ Fine-tuning ကို မပံ့ပိုးနိုင်ပါဘူး - နောက်ဆုံးအချက်အလက်အတွက် [OpenAI documentation](https://platform.openai.com/docs/guides/fine-tuning/what-models-can-be-fine-tuned?WT.mc_id=academic-105485-koreyst) ကိုကြည့်ပါ။ Fine-tuned model တစ်ခုကို ထပ်မံ Fine-tune လုပ်နိုင်ပါသေးတယ်။ ဒီသင်ခန်းစာမှာတော့ Fine-tuning အတွက် `gpt-35-turbo` ကို foundation model အဖြစ် သုံးသွားမှာပါ။

---


### အဆင့် ၁.၁ - သင့်ဒေတာအစုကို ပြင်ဆင်ပါ

အခု ကျွန်တော်တို့ Periodic Table ထဲက ဓာတုဒြပ်စင်တစ်ခုအကြောင်း မေးခွန်းတွေကို limerick နဲ့ ဖြေပြပေးနိုင်တဲ့ chatbot တစ်ခု တည်ဆောက်မယ်။ ဒီလွယ်လွယ်ကူကူ လမ်းညွှန်ထဲမှာတော့ မော်ဒယ်ကို သင်ကြားဖို့အတွက် ရှာဖွေထားတဲ့ data format ကို ပြသပေးနိုင်တဲ့ နမူနာအဖြေ အနည်းငယ်ပါဝင်တဲ့ dataset တစ်ခုသာ ပြုလုပ်သွားမှာပါ။ လက်တွေ့အသုံးပြုမယ့်အခါမှာတော့ နမူနာအဖြေများစွာပါဝင်တဲ့ dataset တစ်ခုလိုအပ်ပါလိမ့်မယ်။ သင့် application domain အတွက် သင့်တော်တဲ့ open dataset တစ်ခုရှိတယ်ဆိုရင်လည်း အဲဒီ dataset ကို အသုံးပြုနိုင်ပါတယ်၊ ဒါမှမဟုတ် format ပြောင်းပြီး fine-tuning အတွက် အသုံးပြုနိုင်ပါတယ်။

ဒီ tutorial မှာတော့ `gpt-35-turbo` ကို အခြေခံပြီး တစ်ကြိမ်တည်းဖြေတဲ့ (single-turn) chat completion ကို ရည်ရွယ်ထားတာမို့ [အကြံပြုထားတဲ့ format](https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset?WT.mc_id=academic-105485-koreyst) ကို အသုံးပြုပြီး OpenAI chat completion လိုအပ်ချက်တွေကို ပြသနိုင်တဲ့ နမူနာတွေ ပြုလုပ်နိုင်ပါတယ်။ မေးခွန်းအကြိမ်ကြီးဖြေတဲ့ (multi-turn) conversation ကို မျှော်လင့်တယ်ဆိုရင်တော့ [multi-turn example format](https://platform.openai.com/docs/guides/fine-tuning/multi-turn-chat-examples?WT.mc_id=academic-105485-koreyst) ကို အသုံးပြုရပါမယ်၊ ဒီမှာတော့ `weight` parameter ပါဝင်ပြီး fine-tuning လုပ်တဲ့အခါ ဘယ် message တွေကို အသုံးပြု/မပြုသင့်တာကို ပြောပြနိုင်ပါတယ်။

ဒီ tutorial မှာတော့ ပိုမိုလွယ်ကူတဲ့ single-turn format ကို သုံးသွားမှာပါ။ ဒေတာတွေက [jsonl format](https://jsonlines.org/?WT.mc_id=academic-105485-koreyst) နဲ့ တစ်ကြောင်းမှာ record တစ်ခုစီပါဝင်ပါတယ်၊ record တစ်ခုစီက JSON format နဲ့ ရေးထားတဲ့ object တစ်ခုပါ။ အောက်က နမူနာမှာ record ၂ ခုကို ပြထားပါတယ် - နောက်ထပ် နမူနာ ၁၀ ခုလုံးကို [training-data.jsonl](../../../../../18-fine-tuning/python/openai/training-data.jsonl) မှာ ကြည့်နိုင်ပါတယ်။ **မှတ်ချက်** - record တစ်ခုစီကို တစ်ကြောင်းတည်းမှာရေးရပါမယ် (JSON file တစ်ခုလို ကြောင်းခွဲရေးတာမျိုး မလုပ်ရပါ)

```json
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Gallium"}, {"role": "assistant", "content": "Gallium, oh gallium, so light - Melts in your hand, oh what a sight - At 86 degrees - Its liquid with ease - And in semiconductors, it's out of sight"}]}
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Hydrogen"}, {"role": "assistant", "content": "Hydrogen, the first in the line - The lightest of all, so divine - It's in water, you see - And in stars, it's the key - The universe's most common sign"}]}
```

လက်တွေ့အသုံးပြုမယ့်အခါမှာတော့ အရည်အသွေးကောင်းတဲ့ရလဒ်ရဖို့ နမူနာအရေအတွက် များများလိုအပ်ပါတယ် - အဖြေ quality နဲ့ fine-tuning လုပ်ဖို့ ကြာချိန်/ကုန်ကျစရိတ်ကြားမှာ tradeoff ရှိပါတယ်။ ဒီ tutorial မှာတော့ fine-tuning လုပ်တဲ့ လုပ်ငန်းစဉ်ကို မြန်မြန်လုပ်ပြနိုင်ဖို့ နမူနာနည်းနည်းနဲ့ သုံးထားတာပါ။ ပိုမိုရှုပ်ထွေးတဲ့ fine-tuning tutorial အတွက် [OpenAI Cookbook နမူနာ](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_finetune_chat_models.ipynb?WT.mc_id=academic-105485-koreyst) ကိုလည်း ကြည့်နိုင်ပါတယ်။


### အဆင့် ၁.၂ သင့်ဒေတာအစုကို တင်ပါ

Files API ကို အသုံးပြုပြီး ဒေတာကို တင်ပါ [ဤနေရာတွင် ဖော်ပြထားသည့်အတိုင်း](https://platform.openai.com/docs/guides/fine-tuning/upload-a-training-file)။ ဤကုဒ်ကို အသုံးပြုရန် မတိုင်မီ အောက်ပါအဆင့်များကို ပြီးစီးထားရပါမည် -
 - `openai` Python package ကို တပ်ဆင်ပြီးဖြစ်ရမည် (နောက်ဆုံးအင်္ဂါရပ်များအတွက် ဗားရှင်း >=0.28.0 ကို အသုံးပြုပါ)
 - သင့် OpenAI API key ကို `OPENAI_API_KEY` environment variable အဖြစ် သတ်မှတ်ပြီးဖြစ်ရမည်
ပိုမိုသိရှိလိုပါက သင်တန်းအတွက် ပံ့ပိုးထားသော [တပ်ဆင်ခြင်း လမ်းညွှန်](./../../../00-course-setup/02-setup-local.md?WT.mc_id=academic-105485-koreyst) ကို ကြည့်ပါ။

ယခု သင့်ရဲ့ ဒေသခံ JSONL ဖိုင်မှ တင်ရန်ဖိုင်တစ်ခု ဖန်တီးရန် ကုဒ်ကို လည်ပတ်ပါ။


In [24]:
from openai import OpenAI
client = OpenAI()

ft_file = client.files.create(
  file=open("./training-data.jsonl", "rb"),
  purpose="fine-tune"
)

print(ft_file)
print("Training File ID: " + ft_file.id)

FileObject(id='file-JdAJcagdOTG6ACNlFWzuzmyV', bytes=4021, created_at=1715566183, filename='training-data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)
Training File ID: file-JdAJcagdOTG6ACNlFWzuzmyV


### အဆင့် ၂.၁ - SDK ကို အသုံးပြု၍ Fine-tuning အလုပ်ကို ဖန်တီးခြင်း


In [25]:
from openai import OpenAI
client = OpenAI()

ft_filejob = client.fine_tuning.jobs.create(
  training_file=ft_file.id, 
  model="gpt-3.5-turbo"
)

print(ft_filejob)
print("Fine-tuning Job ID: " + ft_filejob.id)

FineTuningJob(id='ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', created_at=1715566184, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-EZ6ag0n0S6Zm8eV9BSWKmE6l', result_files=[], seed=830529052, status='validating_files', trained_tokens=None, training_file='file-JdAJcagdOTG6ACNlFWzuzmyV', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)
Fine-tuning Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh


### အဆင့် ၂.၂ - အလုပ်အခြေအနေကို စစ်ဆေးခြင်း

`client.fine_tuning.jobs` API ကို အသုံးပြု၍ ပြုလုပ်နိုင်သောအရာများမှာ အောက်ပါအတိုင်းဖြစ်သည် -
- `client.fine_tuning.jobs.list(limit=<n>)` - နောက်ဆုံး fine-tuning အလုပ် n ခုကို စာရင်းပြုစုသည်
- `client.fine_tuning.jobs.retrieve(<job_id>)` - သတ်မှတ်ထားသော fine-tuning အလုပ်တစ်ခု၏ အသေးစိတ်အချက်အလက်များကို ရယူသည်
- `client.fine_tuning.jobs.cancel(<job_id>)` - fine-tuning အလုပ်တစ်ခုကို ပယ်ဖျက်သည်
- `client.fine_tuning.jobs.list_events(fine_tuning_job_id=<job_id>, limit=<b>)` - အလုပ်တစ်ခုမှ အဖြစ်အပျက်များ n ခုအထိ စာရင်းပြုစုသည်
- `client.fine_tuning.jobs.create(model="gpt-35-turbo", training_file="your-training-file.jsonl", ...)`

ဤလုပ်ငန်းစဉ်၏ ပထမဆုံးအဆင့်မှာ _လေ့ကျင့်သည့်ဖိုင်ကို စစ်ဆေးခြင်း_ ဖြစ်ပြီး၊ ဒေတာသည် မှန်ကန်သောဖော်မတ်ဖြစ်ကြောင်း သေချာစေရန်ဖြစ်သည်။


In [26]:
from openai import OpenAI
client = OpenAI()

# List 10 fine-tuning jobs
client.fine_tuning.jobs.list(limit=10)

# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve(ft_filejob.id)

# List up to 10 events from a fine-tuning job
client.fine_tuning.jobs.list_events(fine_tuning_job_id=ft_filejob.id, limit=10)

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-GkWiDgZmOsuv4q5cSTEGscY6', created_at=1715566184, level='info', message='Validating training file: file-JdAJcagdOTG6ACNlFWzuzmyV', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-3899xdVTO3LN7Q7LkKLMJUnb', created_at=1715566184, level='info', message='Created fine-tuning job: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', object='fine_tuning.job.event', data={}, type='message')], object='list', has_more=False)

In [30]:
# Once the training data is validated
# Track the job status to see if it is running and when it is complete
from openai import OpenAI
client = OpenAI()

response = client.fine_tuning.jobs.retrieve(ft_filejob.id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)

Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh
Status: running
Trained Tokens: None


### အဆင့် ၂.၃ - ဖြစ်စဉ်တိုးတက်မှုကို စောင့်ကြည့်ရန် အဖြစ်အပျက်များကို မှတ်တမ်းတင်ပါ


In [44]:
# You can also track progress in a more granular way by checking for events
# Refresh this code till you get the `The job has successfully completed` message
response = client.fine_tuning.jobs.list_events(ft_filejob.id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Step 85/100: training loss=0.14
Step 86/100: training loss=0.00
Step 87/100: training loss=0.00
Step 88/100: training loss=0.07
Step 89/100: training loss=0.00
Step 90/100: training loss=0.00
Step 91/100: training loss=0.00
Step 92/100: training loss=0.00
Step 93/100: training loss=0.00
Step 94/100: training loss=0.00
Step 95/100: training loss=0.08
Step 96/100: training loss=0.05
Step 97/100: training loss=0.00
Step 98/100: training loss=0.00
Step 99/100: training loss=0.00
Step 100/100: training loss=0.00
Checkpoint created at step 80 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyyF2:ckpt-step-80
Checkpoint created at step 90 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyzhK:ckpt-step-90
New fine-tuned model created: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz
The job has successfully completed


### အဆင့် ၂.၄ - OpenAI ဒက်ရှ်ဘုတ်တွင် အခြေအနေကို ကြည့်ရှုပါ


သင်သည် OpenAI ဝက်ဘ်ဆိုက်သို့ သွားရောက်ကြည့်ရှု၍ _Fine-tuning_ အပိုင်းတွင်လည်း အခြေအနေကိုကြည့်ရှုနိုင်ပါသည်။ ဤနေရာတွင် လက်ရှိလုပ်ဆောင်နေသော အလုပ်၏အခြေအနေကိုကြည့်ရှုနိုင်သလို၊ ယခင်လုပ်ဆောင်ခဲ့သော အလုပ်များ၏ မှတ်တမ်းများကိုလည်း ကြည့်ရှုနိုင်ပါသည်။ ဤ screenshot တွင် ယခင်လုပ်ဆောင်မှုတစ်ခု မအောင်မြင်ခဲ့ပြီး၊ ဒုတိယအကြိမ်မှာ အောင်မြင်ခဲ့သည်ကို တွေ့နိုင်ပါသည်။ နောက်ခံအနေနဲ့ ပြောရရင်၊ ပထမအကြိမ်မှာ JSON ဖိုင်ထဲမှာ မှားယွင်းစွာ ဖော်ပြထားတဲ့ မှတ်တမ်းတွေကြောင့် ဖြစ်ပေါ်ခဲ့တာပါ - ပြင်ဆင်ပြီးနောက် ဒုတိယအကြိမ်မှာ အောင်မြင်စွာ ပြီးမြောက်သွားပြီး မော်ဒယ်ကို အသုံးပြုနိုင်အောင် ဖြစ်လာခဲ့ပါတယ်။

![Fine-tuning job status](../../../../../translated_images/fine-tuned-model-status.563271727bf7bfba7e3f73a201f8712fae3cea1c08f7c7f12ca469c06d234122.my.png)


သင်သည် visual dashboard တွင် အောက်သို့ scroll လုပ်ခြင်းဖြင့်လည်း status messages နှင့် metrics များကို ကြည့်ရှုနိုင်ပါသည်-

| Messages | Metrics |
|:---|:---|
| ![Messages](../../../../../translated_images/fine-tuned-messages-panel.4ed0c2da5ea1313b3a706a66f66bf5007c379cd9219cfb74cb30c0b04b90c4c8.my.png) |  ![Metrics](../../../../../translated_images/fine-tuned-metrics-panel.700d7e4995a652299584ab181536a6cfb67691a897a518b6c7a2aa0a17f1a30d.my.png)|


### အဆင့် ၃.၁ - ID ကို ရယူခြင်းနှင့် Fine-Tuned မော်ဒယ်ကို ကုဒ်ထဲမှာ စမ်းသပ်ကြည့်ခြင်း


In [46]:
# Retrieve the identity of the fine-tuned model once ready
response = client.fine_tuning.jobs.retrieve(ft_filejob.id)
fine_tuned_model_id = response.fine_tuned_model
print("Fine-tuned Model ID:", fine_tuned_model_id)

Fine-tuned Model ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz


In [47]:
# You can then use that model to generate completions from the SDK as shown
# Or you can load that model into the OpenAI Playground (in the UI) to validate it from there.
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model=fine_tuned_model_id,
  messages=[
    {"role": "system", "content": "You are Elle, a factual chatbot that answers questions about elements in the periodic table with a limerick"},
    {"role": "user", "content": "Tell me about Strontium"},
  ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content="Strontium, a metal so bright - It's in fireworks, a dazzling sight - It's in bones, you see - And in tea, it's the key - It's the fortieth, so pure, that's the right", role='assistant', function_call=None, tool_calls=None)


### အဆင့် ၃.၂ - Playground တွင် Fine-Tuned Model ကို Load လုပ်ပြီး စမ်းသပ်ခြင်း

ယခု Fine-tuned model ကို စမ်းသပ်ရန် နည်းလမ်းနှစ်ခုရှိသည်။ ပထမနည်းမှာ Playground ကို သွားပြီး Models drop-down မှာ သင့် Fine-tuned model ကို ရွေးချယ်နိုင်သည်။ နောက်ထပ်နည်းတစ်ခုကတော့ Fine-tuning panel (အထက်ပါ screenshot တွင်ပြထားသည်) မှာ "Playground" ကိုနှိပ်ခြင်းဖြင့် _နှိုင်းယှဉ်ကြည့်နိုင်သော_ view ကို ဖွင့်ပေးမည်ဖြစ်ပြီး၊ foundation model နဲ့ fine-tuned model နှစ်ခုလုံးကို တစ်ပြိုင်နက်တည်း မြင်နိုင်သဖြင့် အလွယ်တကူ စမ်းသပ်နိုင်သည်။

![Fine-tuning job status](../../../../../translated_images/fine-tuned-playground-compare.56e06f0ad8922016497d39ced3d84ea296eec89073503f2bf346ec9718f913b5.my.png)

သင့် training data တွင်သုံးထားသော system context ကို ဖြည့်ပြီး စမ်းသပ်မည့်မေးခွန်းကို ထည့်ပေးပါ။ နှစ်ဖက်လုံးတွင် context နဲ့ မေးခွန်းတူညီစွာ ပြသထားသည်ကို တွေ့ရမည်။ နှိုင်းယှဉ်မှုကို run လုပ်ပြီးနောက် output များအကြား ကွာခြားချက်ကို မြင်နိုင်ပါလိမ့်မည်။ _Fine-tuned model သည် သင်ပေးထားသော ဥပမာများအတိုင်း format ကိုလိုက်နာပြီး ဖြေကြားပေးသော်လည်း foundation model သည် system prompt ကိုသာလိုက်နာသည်ကို သတိပြုကြည့်ပါ_။

![Fine-tuning job status](../../../../../translated_images/fine-tuned-playground-launch.5a26495c983c6350c227e05700a47a89002d132949a56fa4ff37f266ebe997b2.my.png)

နှိုင်းယှဉ်မှုတွင် model တစ်ခုချင်းစီအတွက် token အရေအတွက်နှင့် inference အတွက် ကြာချိန်ကိုလည်း ပြသပေးသည်ကို တွေ့ရမည်။ **ဤဥပမာသည် လုပ်ငန်းစဉ်ကို ပြသရန်သာ ရည်ရွယ်ထားသော်လည်း တကယ့်လောကအတွက် dataset သို့မဟုတ် အခြေအနေတစ်ခုကို မဖော်ပြပါ**။ နှစ်ခုလုံးတွင် token အရေအတွက်တူညီသော်လည်း (system context နဲ့ user prompt တူညီသည်) fine-tuned model (custom model) သည် inference အတွက် ပိုကြာသည်ကို တွေ့နိုင်သည်။

တကယ့်လောကအခြေအနေများတွင် ဤလို ဥပမာလေးများမသုံးတော့ဘဲ တကယ့် data (ဥပမာ- ဖောက်သည်ဝန်ဆောင်မှုအတွက် ပစ္စည်းစာရင်း) များဖြင့် fine-tune လုပ်မည်ဖြစ်ပြီး၊ ဖြေကြားမှုအရည်အသွေးက ပိုမိုရှင်းလင်းစွာ ထင်ရှားလာမည်။ ထိုအခြေအနေတွင် foundation model ဖြင့် တူညီသော ဖြေကြားမှုအရည်အသွေးရရှိရန် custom prompt engineering ပိုမိုလုပ်ရမည်ဖြစ်ပြီး၊ ထို့ကြောင့် token အသုံးအများနှင့် inference processing time တိုးလာနိုင်သည်။ _ဤအရာကို စမ်းသပ်လိုပါက OpenAI Cookbook ထဲရှိ fine-tuning ဥပမာများကို ကြည့်ပြီး စတင်နိုင်သည်_။



---

**သတိပြုရန်**:
ဤစာရွက်စာတမ်းကို AI ဘာသာပြန်ဝန်ဆောင်မှု [Co-op Translator](https://github.com/Azure/co-op-translator) ကို အသုံးပြု၍ ဘာသာပြန်ထားပါသည်။ ကျွန်ုပ်တို့သည် တိကျမှန်ကန်မှုအတွက် ကြိုးစားနေသော်လည်း၊ အလိုအလျောက် ဘာသာပြန်ချက်များတွင် အမှားများ သို့မဟုတ် မမှန်ကန်မှုများ ပါဝင်နိုင်သည်ကို သတိပြုပါ။ မူရင်းဘာသာဖြင့် ရေးသားထားသော စာရွက်စာတမ်းကို အာဏာတရားရှိသော ရင်းမြစ်အဖြစ် ယူဆသင့်ပါသည်။ အရေးကြီးသော အချက်အလက်များအတွက် လူ့ဘာသာပြန်ပညာရှင်များ၏ ဝန်ဆောင်မှုကို အသုံးပြုရန် အကြံပြုပါသည်။ ဤဘာသာပြန်ချက်ကို အသုံးပြုခြင်းမှ ဖြစ်ပေါ်လာသော နားလည်မှုမှားခြင်း သို့မဟုတ် အနားလည်အမှားများအတွက် ကျွန်ုပ်တို့သည် တာဝန်မယူပါ။
